In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.data import shepp_logan_phantom
from skimage.transform import radon, iradon
from ipywidgets import interact, IntSlider, Dropdown, interactive_output

# Generate the Shepp-Logan phantom image
phantom = shepp_logan_phantom()
phantom = phantom[::2, ::2]  # Resize to make it faster for demonstration

# Function to perform Radon transform, simulate projections, and reconstruction
def simulate_reconstruction(num_angles, filter_type, selected_projection):
    # Generate projection angles
    theta = np.linspace(0., 180., num_angles, endpoint=False)
    
    # Radon transform (simulating projections)
    sinogram = radon(phantom, theta=theta, circle=True)
    
    # Reconstruction using Filtered Back Projection (FBP)
    reconstructed_image = iradon(sinogram, theta=theta, filter_name=filter_type, circle=True)
    
    # Plot the results
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    ax = axes.ravel()
    
    # Plot the original phantom
    ax[0].set_title("Original Phantom")
    ax[0].imshow(phantom, cmap='gray')
    ax[0].axis('off')
    
    # Plot the sinogram with a vertical line indicating the selected projection
    ax[1].set_title(f"Sinogram (Angles: {num_angles})")
    ax[1].imshow(sinogram, cmap='gray', aspect='auto')
    ax[1].axvline(x=selected_projection, color='red', linestyle='--')
    ax[1].set_xlabel("Projection angle (index)")
    ax[1].set_ylabel("Projection position (pixels)")
    
    # Plot the reconstructed image
    ax[2].set_title(f"Reconstruction\n(Filter: {filter_type})")
    ax[2].imshow(reconstructed_image, cmap='gray')
    ax[2].axis('off')
    
    # Plot the selected projection line
    projection_data = sinogram[:, selected_projection]
    ax[3].set_title(f"Projection at Angle {theta[selected_projection]:.1f}°")
    ax[3].plot(projection_data, color='blue')
    ax[3].set_xlabel("Position (pixels)")
    ax[3].set_ylabel("Intensity")
    
    plt.tight_layout()
    plt.show()

# Interactive widgets with dynamic slider for selected_projection
num_angles_slider = IntSlider(value=30, min=1, max=256, step=1, description="Number of Angles")
filter_type_dropdown = Dropdown(
    options=["ramp", "shepp-logan", "cosine", "hamming", "hann", None],
    value="ramp",
    description="FBP Filter"
)

# Function to update the max value of selected_projection slider
def update_selected_projection_slider(num_angles):
    selected_projection_slider.max = max(0, num_angles - 1)

# Initialize selected_projection slider
selected_projection_slider = IntSlider(value=0, min=0, max=29, step=1, description="Projection Index")

# Set up the interactivity with dynamic dependency
interactive_plot = interactive_output(
    simulate_reconstruction,
    {'num_angles': num_angles_slider,
     'filter_type': filter_type_dropdown,
     'selected_projection': selected_projection_slider}
)

# Link num_angles to selected_projection max update
num_angles_slider.observe(lambda change: update_selected_projection_slider(change['new']), names='value')

# Display all widgets and output
display(num_angles_slider, filter_type_dropdown, selected_projection_slider, interactive_plot)
